# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770997261210                   -0.53    9.0         
  2   -2.772149888801       -2.94       -1.32    1.0    161ms
  3   -2.772170386313       -4.69       -2.51    1.0    180ms
  4   -2.772170667329       -6.55       -3.29    1.0    164ms
  5   -2.772170720793       -7.27       -3.83    2.0    183ms
  6   -2.772170722793       -8.70       -4.41    1.0    183ms
  7   -2.772170723008       -9.67       -5.13    2.0    187ms
  8   -2.772170723015      -11.17       -6.10    1.0    181ms
  9   -2.772170723015      -12.70       -6.73    2.0    191ms
 10   -2.772170723015   +  -14.03       -6.75    1.0    182ms
 11   -2.772170723015      -14.05       -7.62    1.0    191ms
 12   -2.772170723015      -13.88       -8.40    2.0    204ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735580511009765

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770748881673                   -0.53    9.0         
  2   -2.772052865024       -2.88       -1.31    1.0    161ms
  3   -2.772083036702       -4.52       -2.56    1.0    163ms
  4   -2.772083389070       -6.45       -3.60    1.0    189ms
  5   -2.772083416494       -7.56       -3.97    2.0    184ms
  6   -2.772083417742       -8.90       -4.87    1.0    173ms
  7   -2.772083417807      -10.19       -5.27    2.0    217ms
  8   -2.772083417810      -11.49       -5.74    1.0    173ms
  9   -2.772083417811      -12.19       -6.27    2.0    198ms
 10   -2.772083417811      -13.37       -7.41    1.0    193ms
 11   -2.772083417811   +  -13.92       -7.86    2.0    192ms
 12   -2.772083417811      -15.35       -8.24    1.0    200ms

Polarizability via ForwardDiff:       1.7725349848214556
Polarizability via finite difference: 1.7735580511009765
